In [ ]:
import csv

# Örnek veri seti 5 örnek 3 cümle
data = [
    {"Sentence 1": "The cat is sitting on the mat.", 
     "Sentence 2": "A cat is resting on a rug.", 
     "Sentence 3": "The car is driving down the road."},
    {"Sentence 1": "Birds are flying in the sky.", 
     "Sentence 2": "A flock of birds is moving through the air.", 
     "Sentence 3": "It is raining heavily outside."},
    {"Sentence 1": "The dog is barking loudly in the yard.", 
     "Sentence 2": "A dog barks at the stranger.", 
     "Sentence 3": "The sun is shining brightly."},
    {"Sentence 1": "Children are playing soccer in the park.", 
     "Sentence 2": "Kids are kicking the ball around.", 
     "Sentence 3": "A tree is swaying in the wind."},
    {"Sentence 1": "A train is arriving at the station.", 
     "Sentence 2": "The locomotive is pulling into the platform.", 
     "Sentence 3": "Passengers are waiting in line."}
]

# CSV dosyasına yazma
with open("dataset.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["Sentence 1", "Sentence 2", "Sentence 3"])
    writer.writeheader()  # Başlıkları yaz
    writer.writerows(data)  # Verileri yaz

print("Veri seti 'dataset.csv' olarak kaydedildi.")


Veri seti 'dataset.csv' olarak kaydedildi.


In [6]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"


In [8]:
from transformers import AutoTokenizer, AutoModel
import torch

# BERT modeli ve tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Embedding fonksiyonu
def get_embeddings(text, method="cls"):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    if method == "cls":
        return outputs.last_hidden_state[:, 0, :].detach().numpy()  # [CLS] token
    elif method == "pooled":
        return outputs.pooler_output.detach().numpy()  # Pooled output


In [9]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [10]:
from sentence_transformers import SentenceTransformer

# Sentence-BERT modeli
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Embedding fonksiyonu
def get_sbert_embeddings(text):
    return sbert_model.encode(text)

In [ ]:
pip install sentence_transformers #kütüphane indirdim


Note: you may need to restart the kernel to use updated packages.


In [15]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Kosinüs benzerliğini hesaplama fonksiyonu
def calculate_similarity(sentence1, sentence2, method="cls"):
    if method in ["cls", "pooled"]:
        vec1 = get_embeddings(sentence1, method)
        vec2 = get_embeddings(sentence2, method)
    elif method == "sbert":
        vec1 = get_sbert_embeddings(sentence1)
        vec2 = get_sbert_embeddings(sentence2)
    similarity = cosine_similarity([vec1.flatten()], [vec2.flatten()])
    return similarity[0][0]


In [ ]:
results = []

for item in data:
    s1, s2, s3 = item["Sentence 1"], item["Sentence 2"], item["Sentence 3"]

    #[CLS] Token (için cümleler arası benzerlikleri hesaplamak için calculate_similarity fonksiyonu kullandım)
    cls_sim_12 = calculate_similarity(s1, s2, method="cls")
    cls_sim_13 = calculate_similarity(s1, s3, method="cls")
    cls_sim_23 = calculate_similarity(s2, s3, method="cls")

    # Pooled Embeddings (için cümleler arası benzerlikleri hesaplamak için calculate_similarity fonksiyonu kullandım)
    pooled_sim_12 = calculate_similarity(s1, s2, method="pooled")
    pooled_sim_13 = calculate_similarity(s1, s3, method="pooled")
    pooled_sim_23 = calculate_similarity(s2, s3, method="pooled")

    # Sentence-BERT  (için cümleler arası benzerlikleri hesaplamak için calculate_similarity fonksiyonu kullandım)
    sbert_sim_12 = calculate_similarity(s1, s2, method="sbert")
    sbert_sim_13 = calculate_similarity(s1, s3, method="sbert")
    sbert_sim_23 = calculate_similarity(s2, s3, method="sbert")

    results.append({
        "Sentences": (s1, s2, s3),
        "CLS": (cls_sim_12, cls_sim_13, cls_sim_23),
        "Pooled": (pooled_sim_12, pooled_sim_13, pooled_sim_23),
        "SBERT": (sbert_sim_12, sbert_sim_13, sbert_sim_23)
    })

print("Sonuçlar başarıyla hesaplandı.")
for result in results:
    print(result)


Sonuçlar başarıyla hesaplandı.
{'Sentences': ('The cat is sitting on the mat.', 'A cat is resting on a rug.', 'The car is driving down the road.'), 'CLS': (0.84120286, 0.93609846, 0.8091303), 'Pooled': (0.9866842, 0.99512213, 0.9883174), 'SBERT': (0.71481824, 0.1373416, 0.13539922)}
{'Sentences': ('Birds are flying in the sky.', 'A flock of birds is moving through the air.', 'It is raining heavily outside.'), 'CLS': (0.95398337, 0.87952065, 0.88785815), 'Pooled': (0.9978446, 0.9897523, 0.9883157), 'SBERT': (0.74088526, 0.1983135, 0.1941022)}
{'Sentences': ('The dog is barking loudly in the yard.', 'A dog barks at the stranger.', 'The sun is shining brightly.'), 'CLS': (0.9311013, 0.9393015, 0.8986623), 'Pooled': (0.99086064, 0.9887873, 0.9714521), 'SBERT': (0.502006, 0.09380062, 0.0595571)}
{'Sentences': ('Children are playing soccer in the park.', 'Kids are kicking the ball around.', 'A tree is swaying in the wind.'), 'CLS': (0.92700255, 0.9136472, 0.89860594), 'Pooled': (0.9972928, 0

In [ ]:
pip install data  #datayı built etmek içi kullandım

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Ortalama benzerlik hesaplama(burada avg_sim methou kullanarak yaptım 1-2, 1-3, 2-3 Sentence 1 2 ve 3 arasındaki benzerlik bakıyor. )
def average_similarity(results, method):
    sim_12 = np.mean([r[method][0] for r in results])
    sim_13 = np.mean([r[method][1] for r in results])
    sim_23 = np.mean([r[method][2] for r in results])
    return sim_12, sim_13, sim_23

methods = ["CLS", "Pooled", "SBERT"]
overall_averages = {method: average_similarity(results, method) for method in methods}

for method in methods:
    avg_sim_12, avg_sim_13, avg_sim_23 = overall_averages[method]
    print(f"{method} için ortalama benzerlik:")
    print(f"  Sentence 1 & Sentence 2: {avg_sim_12:.4f}")
    print(f"  Sentence 1 & Sentence 3: {avg_sim_13:.4f}")
    print(f"  Sentence 2 & Sentence 3: {avg_sim_23:.4f}")
    print()


CLS için ortalama benzerlik:
  Sentence 1 & Sentence 2: 0.9208
  Sentence 1 & Sentence 3: 0.8964
  Sentence 2 & Sentence 3: 0.8666

Pooled için ortalama benzerlik:
  Sentence 1 & Sentence 2: 0.9937
  Sentence 1 & Sentence 3: 0.9926
  Sentence 2 & Sentence 3: 0.9878

SBERT için ortalama benzerlik:
  Sentence 1 & Sentence 2: 0.6328
  Sentence 1 & Sentence 3: 0.1913
  Sentence 2 & Sentence 3: 0.1242



In [ ]:
## Kendi yorumum;

#CLS: Cümleler arasında yeterince ayrım yapamamış.

#Pooled Embeddings: Tüm çiftler için aşırı yüksek benzerlik değerleri ürettiği için anlam farklarını yakalayamamış.

#S-BERT: Benzer olan ve olmayan cümleler arasında en net ayrımı yapan yöntemdir. Bu yüzden en iyi çalışan yöntem S-BERT'dir.


#Not: S-BERT’in düşük skorlar vermesi kötü bir sonuç gibi görünebilir, 
# ancak burada önemli olan, benzer cümleler için yüksek, farklı cümleler için düşük skorlar verebilmesidir. Bu görevi en iyi şekilde başarmış.